# 沪深300指数成分和权重获取

本notebook演示如何使用Tushare API获取沪深300指数的成分股和权重信息。

根据 `docs/指数成分和权重.md` 的说明：
- 接口：index_weight
- 描述：获取各类指数成分和权重，**月度数据**
- 建议输入参数里开始日期和结束日分别输入当月第一天和最后一天的日期
- 用户需要至少2000积分才可以调取


In [2]:
# 导入必要的模块
import sys
import os
from datetime import datetime, date
from pathlib import Path

# 由于现在在data目录下，需要添加项目根目录到路径
current_dir = Path.cwd()
project_root = current_dir.parent
sys.path.append(str(project_root))

print(f"当前目录: {current_dir}")
print(f"项目根目录: {project_root}")
print(f"Python路径: {sys.path[-1]}")

import pandas as pd
import tushare as ts
from src.tools.utils import _init_tushare_api

当前目录: /Users/wuye/Desktop/gjzq_intern/project_code/0926/BacktestAgent/data
项目根目录: /Users/wuye/Desktop/gjzq_intern/project_code/0926/BacktestAgent
Python路径: /Users/wuye/Desktop/gjzq_intern/project_code/0926/BacktestAgent


## 1. 初始化Tushare API

首先需要确保TUSHARE_TOKEN环境变量已设置。


In [3]:
# 初始化Tushare API
try:
    pro = _init_tushare_api()
    print("Tushare API 初始化成功")
    print(f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
except Exception as e:
    print(f"Tushare API 初始化失败: {e}")
    print("请确保已设置TUSHARE_TOKEN环境变量")


Tushare API 初始化成功
当前时间: 2025-09-29 15:22:58


## 2. 获取沪深300指数成分和权重

根据文档说明和用户要求：
- 沪深300指数代码：000300.SZ  
- 使用指定的交易日期：20250901

In [3]:
# 设置查询参数
trade_date = '20250901'  # 指定交易日期
index_code = '000300.SH'  # 沪深300指数代码 (修改为000300)

print(f"查询参数:")
print(f"指数代码: {index_code}")
print(f"交易日期: {trade_date}")

查询参数:
指数代码: 000300.SH
交易日期: 20250901


In [4]:
# 获取沪深300指数成分和权重
try:
    print(f"正在获取 {index_code} 指数成分和权重...")
    df = pro.index_weight(
        index_code=index_code,
        trade_date=trade_date
    )
    
    print(f"成功获取数据，共 {len(df)} 条记录")
    print("\n数据样例（前10条）:")
    print(df.head(10))
    
except Exception as e:
    print(f"获取数据失败: {e}")
    print("可能的原因:")
    print("1. 积分不足（需要至少2000积分）") 
    print("2. 网络连接问题")
    print("3. 参数错误")
    print("4. 指定日期无数据")

正在获取 000300.SH 指数成分和权重...
成功获取数据，共 300 条记录

数据样例（前10条）:
  index_code   con_code trade_date  weight
0  000300.SH  600519.SH   20250901  3.8616
1  000300.SH  300750.SZ   20250901  3.3453
2  000300.SH  601318.SH   20250901  2.6228
3  000300.SH  600036.SH   20250901  2.1642
4  000300.SH  601899.SH   20250901  1.6725
5  000300.SH  300502.SZ   20250901  1.6057
6  000300.SH  000333.SZ   20250901  1.5166
7  000300.SH  300059.SZ   20250901  1.5108
8  000300.SH  601166.SH   20250901  1.5097
9  000300.SH  300308.SZ   20250901  1.5035


## 3. 数据分析和处理


In [5]:
# 数据基本信息
if 'df' in locals() and not df.empty:
    print("数据基本信息:")
    print(f"总记录数: {len(df)}")
    print(f"唯一成分股数量: {df['con_code'].nunique()}")
    print(f"交易日期范围: {df['trade_date'].min()} 到 {df['trade_date'].max()}")
    
    print("\n权重统计:")
    print(df['weight'].describe())
    
    # 显示数据结构
    print("\n数据结构:")
    print(df.dtypes)
    
    # 检查是否有重复数据
    duplicates = df.duplicated().sum()
    print(f"\n重复记录数: {duplicates}")


数据基本信息:
总记录数: 300
唯一成分股数量: 300
交易日期范围: 20250901 到 20250901

权重统计:
count    300.000000
mean       0.333332
std        0.438675
min        0.010500
25%        0.117400
50%        0.190500
75%        0.366525
max        3.861600
Name: weight, dtype: float64

数据结构:
index_code     object
con_code       object
trade_date     object
weight        float64
dtype: object

重复记录数: 0


In [6]:
# 查看权重最高的前10只成分股
if 'df' in locals() and not df.empty:
    print("权重最高的前10只成分股:")
    
    # 获取最新交易日的数据
    latest_date = df['trade_date'].max()
    latest_df = df[df['trade_date'] == latest_date].copy()
    
    # 按权重排序
    top_stocks = latest_df.nlargest(10, 'weight')
    
    print(f"\n最新交易日 ({latest_date}) 权重排名:")
    for i, (_, row) in enumerate(top_stocks.iterrows(), 1):
        print(f"{i:2d}. {row['con_code']:10s} 权重: {row['weight']:6.4f}%")


权重最高的前10只成分股:

最新交易日 (20250901) 权重排名:
 1. 600519.SH  权重: 3.8616%
 2. 300750.SZ  权重: 3.3453%
 3. 601318.SH  权重: 2.6228%
 4. 600036.SH  权重: 2.1642%
 5. 601899.SH  权重: 1.6725%
 6. 300502.SZ  权重: 1.6057%
 7. 000333.SZ  权重: 1.5166%
 8. 300059.SZ  权重: 1.5108%
 9. 601166.SH  权重: 1.5097%
10. 300308.SZ  权重: 1.5035%


## 4. 数据保存

将获取的数据保存到data目录下的CSV文件。

In [ ]:
# 保存数据到文件
if 'df' in locals() and not df.empty:
    # 创建data目录（当前就在data目录下）
    data_dir = Path('.')
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # 保存为CSV文件
    csv_filename = f"hs300_index_weight_{trade_date}.csv"
    csv_path = data_dir / csv_filename
    df.to_csv(csv_path, index=False, encoding='utf-8')
    
    print(f"数据已保存:")
    print(f"CSV文件: {csv_path}")
    print(f"文件大小: {csv_path.stat().st_size / 1024:.2f} KB")
    print(f"保存记录数: {len(df)}")
else:
    print("没有数据可保存")

数据已保存:
CSV文件: ../data/hs300_index_weight_20250901.csv
文件大小: 10.55 KB
保存记录数: 300


## 总结

本notebook展示了如何：

1. 使用项目中已配置的Tushare API工具
2. 获取沪深300指数的成分股和权重数据
3. 对数据进行基本分析和统计
4. 保存数据到本地文件

**注意事项：**
- 需要Tushare Pro账户且积分不少于2000分
- 数据为月度数据，建议按月查询
- 权重数据可能存在调整，请以最新数据为准
